In [1]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize

In [2]:
def sigma_z():
    """
    Create the Pauli-Z matrix
    """
    return np.array([[1, 0], [0, -1]])

def rotation_matrix(phi):
    """
    Create the rotation matrix R(ϕ) = cos(ϕ)X + sin(ϕ)Y
    """
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1j], [1j, 0]])
    return np.cos(phi) * sigma_x + np.sin(phi) * sigma_y

def U(phi, alpha):
    """
    Construct the unitary operator U(ϕ, α) = e^(-iαR(ϕ)/2)
    """
    R_phi = rotation_matrix(phi)
    return expm(-1j * alpha * R_phi / 2)

def concatenate_unitaries(params, N):
    """
    Concatenate N unitary matrices using the given parameters.
    """
    concatenated = np.eye(2)
    for i in range(N):
        phi, alpha = params[i*2], params[i*2+1]
        concatenated = np.dot(U(phi, alpha), concatenated)
    return concatenated

def gate_fom(U, U_target):
    """
    Cost function measuring the distance between the concatenated unitary and the target unitary.
    """
    dim = U.shape[0]
    return np.abs(np.trace(U_target.T.conj() @ U))**2 / dim**2
    

def cost_function(params, target_unitary1, target_unitary2, scaling_factor, double=True):
    """
    Cost function measuring the distance between the concatenated unitary and the target unitary.
    """
    N = len(params) // 2
    concatenated_unitary = concatenate_unitaries(params, N)
    scaled_params = params.copy()
    scaled_params[1::2] *= scaling_factor
    scaled_unitary = concatenate_unitaries(scaled_params, N)
    if double:
        return np.linalg.norm(concatenated_unitary - target_unitary1, 'fro') + np.linalg.norm(scaled_unitary - target_unitary2, 'fro')



def cost_function_hadamard(params, target_unitary1, target_unitary2, scaling_factor, double=True):
    """
    Cost function measuring the distance between the concatenated unitary and the target unitary.
    """
    N = len(params) // 2
    concatenated_unitary = concatenate_unitaries(params, N)
    scaled_params = params.copy()
    scaled_params[1::2] *= scaling_factor
    scaled_unitary = concatenate_unitaries(scaled_params, N)
    fin1 = concatenated_unitary.T.conj()@sigma_z()@concatenated_unitary
    fin2 = scaled_unitary.T.conj()@sigma_z()@scaled_unitary
    if double:
        return np.linalg.norm(fin1 - target_unitary1, 'fro') + np.linalg.norm(fin2 - target_unitary2, 'fro')

def constraint(params, target_unitary, scaling_factor):
    """
    Constraint function checking if concatenation with scaled alpha also reaches the target unitary.
    """
    N = len(params) // 2
    scaled_params = params.copy()
    scaled_params[1::2] *= scaling_factor
    scaled_unitary = concatenate_unitaries(scaled_params, N)
    return np.linalg.norm(scaled_unitary - target_unitary, 'fro')
    #dim = scaled_unitary.shape[0]
    #return np.abs(np.trace(target_unitary.T.conj() @ scaled_unitary))**2 / dim**2 - 1


# Hadamard

In [19]:
N = 4 # number of concatenated unitaries
params = np.array([0, np.pi/4, np.pi/2, np.pi/2, 7/4*np.pi, 3/4 * np.pi, np.pi/4, 3/2*np.pi])
U_target_1 = concatenate_unitaries(params, N)
print("U target for normal atoms\n",np.round(U_target_1, 3))

S = 4
scaling_factor = np.sqrt(S)
scaled_params = params.copy()
scaled_params[1::2] *= scaling_factor
U_target_2 = concatenate_unitaries(scaled_params, N)
print("U target for Super atoms\n",np.round(U_target_2, 3))

U target for normal atoms
 [[-0.707+0.707j -0.   -0.j   ]
 [-0.   -0.j    -0.707-0.707j]]
U target for Super atoms
 [[ 0.146-0.354j -0.354+0.854j]
 [ 0.354+0.854j  0.146+0.354j]]


In [20]:
fin1 = U_target_1.T.conj()@sigma_z()@U_target_1
print("Final state for normal atoms\n",np.round(fin1, 3))

fin2 = U_target_2.T.conj()@sigma_z()@U_target_2
print("Final state for Super atoms\n",np.round(fin2, 3))

Final state for normal atoms
 [[ 1.+0.j -0.+0.j]
 [-0.-0.j -1.+0.j]]
Final state for Super atoms
 [[-0.707+0.j -0.707-0.j]
 [-0.707+0.j  0.707+0.j]]


In [23]:
# Example usage:
# Define the target unitaries
N = 3  # Number of unitaries to concatenate

# Define the target unitaries
target_unitary_1 = fin1
target_unitary_2 = fin2
S = 4
scaling_factor = np.sqrt(S)

#initial_params = np.array([0, np.pi/4, np.pi/2, np.pi/2, 7/4*np.pi, 3/4 * np.pi, np.pi/4, 3/2*np.pi])

# Define optimization bounds
bounds = [(0, 2*np.pi) for _ in range(2*N)]

iterations = 20

# Perform optimization
params_best = np.zeros(2*N)
for i in range(iterations):
    # Initialize parameters randomly
    initial_params = np.random.rand(2*N) * 2 * np.pi
    result = minimize(cost_function_hadamard, initial_params, args=(target_unitary_1, target_unitary_2, scaling_factor, True), bounds=bounds, method='SLSQP')
    cf_best = cost_function_hadamard(params_best, target_unitary_1, target_unitary_2, scaling_factor, True)
    print("Best cost function value:", cf_best)
    if result.fun < cf_best:
        params_best = result.x
    

print("\nOptimized parameters:", params_best)

# Truncate the final matrix elements to three digits
optimized_unitary = concatenate_unitaries(params_best, N)
fin1_exp = optimized_unitary.T.conj()@sigma_z()@optimized_unitary
print("Final state for normal atoms\n",np.round(fin1_exp, 3))

# Test if the scaled corresponding unitary is close to the target unitary
scaled_params = params_best.copy()
scaled_params[1::2] *= scaling_factor
scaled_unitary = concatenate_unitaries(scaled_params, N)
fin2_exp = scaled_unitary.T.conj()@sigma_z()@scaled_unitary
print("Final state for super atoms\n",np.round(fin2_exp, 3))

print("optimized_unitary\n",np.round(optimized_unitary, 3))
print("scaled_unitary\n",np.round(scaled_unitary, 3))


Best cost function value: 2.613125929752754
Best cost function value: 1.0880146567150373e-05
Best cost function value: 9.90817207425176e-06
Best cost function value: 9.90817207425176e-06
Best cost function value: 9.90817207425176e-06
Best cost function value: 9.90817207425176e-06
Best cost function value: 9.90817207425176e-06
Best cost function value: 9.90817207425176e-06
Best cost function value: 9.90817207425176e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06
Best cost function value: 1.9555069563240655e-06

Optimized parameters: [1.62498053 0.90

In [24]:
# Truncate the final matrix elements to three digits
optimized_unitary = concatenate_unitaries(params_best, N)
fin1_exp = optimized_unitary.T.conj()@sigma_z()@optimized_unitary
print("Final state for normal atoms\n",np.round(fin1_exp, 3))

# Test if the scaled corresponding unitary is close to the target unitary
scaled_params = params_best.copy()
scaled_params[1::2] *= scaling_factor
scaled_unitary = concatenate_unitaries(scaled_params, N)
fin2_exp = scaled_unitary.T.conj()@sigma_z()@scaled_unitary
print("Final state for super atoms\n",np.round(fin2_exp, 3))

print("optimized_unitary\n",np.round(optimized_unitary, 3))
print("scaled_unitary\n",np.round(scaled_unitary, 3))


Final state for normal atoms
 [[ 1.+0.j -0.+0.j]
 [-0.-0.j -1.+0.j]]
Final state for super atoms
 [[-0.707+0.j -0.707-0.j]
 [-0.707+0.j  0.707+0.j]]
optimized_unitary
 [[-0.806+0.593j  0.   -0.j   ]
 [-0.   -0.j    -0.806-0.593j]]
scaled_unitary
 [[ 0.233-0.304j -0.563+0.733j]
 [ 0.563+0.733j  0.233+0.304j]]


# CZ 

In [80]:
N = 5 # number of concatenated unitaries
params = np.array([np.pi/2, np.pi/4, 0, np.pi, np.pi/2, np.pi/2, 0, np.pi, np.pi/2, np.pi/4])
U_target_1 = concatenate_unitaries(params, N)
print("U target for normal atoms\n",np.round(U_target_1, 3))

S = 4
scaling_factor = np.sqrt(S)
scaled_params = params.copy()
scaled_params[1::2] *= scaling_factor
U_target_2 = concatenate_unitaries(scaled_params, N)
print("U target for Super atoms\n",np.round(U_target_2, 3))

U target for normal atoms
 [[-1.+0.j  0.+0.j]
 [-0.+0.j -1.-0.j]]
U target for Super atoms
 [[-1.-0.j -0.-0.j]
 [ 0.-0.j -1.+0.j]]


In [86]:
# Example usage:
# Define the target unitaries
N = 4  # Number of unitaries to concatenate

# Define the target unitaries
target_unitary_1 = U_target_1
target_unitary_2 = U_target_2
S = 2
scaling_factor = np.sqrt(S)

#initial_params = np.array([0, np.pi/4, np.pi/2, np.pi/2, 7/4*np.pi, 3/4 * np.pi, np.pi/4, 3/2*np.pi])

# Define optimization bounds
bounds = [(0, 2*np.pi) for _ in range(2*N)]

iterations = 10

# Perform optimization
params_best = np.zeros(2*N)
for i in range(iterations):
    # Initialize parameters randomly
    initial_params = np.random.rand(2*N) * 2 * np.pi
    result = minimize(cost_function, initial_params, args=(target_unitary_1, target_unitary_2, scaling_factor, True), bounds=bounds, method='SLSQP')
    cf_best = cost_function(params_best, target_unitary_1, target_unitary_2, scaling_factor, True)
    print("Best cost function value:", cf_best)
    if result.fun < cf_best:
        params_best = result.x
    

print("\nOptimized parameters:", params_best)

# Truncate the final matrix elements to three digits
optimized_unitary = concatenate_unitaries(params_best, N)

# Test if the scaled corresponding unitary is close to the target unitary
scaled_params = params_best.copy()
scaled_params[1::2] *= scaling_factor
scaled_unitary = concatenate_unitaries(scaled_params, N)

print("optimized_unitary\n",np.round(optimized_unitary, 3))
print("scaled_unitary\n",np.round(scaled_unitary, 3))


Best cost function value: 5.65685424949238
Best cost function value: 1.0131499154387784
Best cost function value: 1.0131499154387784
Best cost function value: 1.0131499154387784
Best cost function value: 1.0119048441359262
Best cost function value: 1.0119048441359262
Best cost function value: 1.3005344141935443e-05
Best cost function value: 1.3005344141935443e-05
Best cost function value: 8.53532802707991e-06
Best cost function value: 8.53532802707991e-06

Optimized parameters: [4.88458039 4.79727363 0.62399415 2.37883696 4.8845832  4.79727286
 0.62399178 2.37882913]
optimized_unitary
 [[-1.+0.j  0.+0.j]
 [-0.+0.j -1.-0.j]]
scaled_unitary
 [[-1.-0.j  0.+0.j]
 [-0.+0.j -1.+0.j]]
